In [1]:
import requests
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv, find_dotenv

In [2]:
# load env data from .env file.
load_dotenv(find_dotenv(filename='.env'))

True

In [3]:
sql = '''
    SELECT * 
    FROM cities;
    '''

con = f'mysql+pymysql://{os.environ["DB_USER"]}:{os.environ["DB_PASSWORD"]}@{os.environ["DB_HOST"]}:{os.environ["DB_PORT"]}/{os.environ["DB_SCHEMA"]}'
cities = pd.read_sql(sql, con)

In [4]:
cities

,city_id,city_name,city_country,city_longitude,city_latitude,municipality_country,created_at,city_pop
0,3247449,Aachen,DE,6.08342,50.7766,"Aachen,DE",2022-04-07 10:32:48,248960
1,2959927,Aalen,DE,10.09330,48.8378,"Aalen,DE",2022-04-07 10:32:48,68361
2,2959855,Abenberg,DE,10.96400,49.2428,"Abenberg,DE",2022-04-07 10:32:48,5511
3,2959842,Abensberg,DE,11.85000,48.8167,"Abensberg,DE",2022-04-07 10:32:48,13946
4,2959686,Achern,DE,8.07607,48.6311,"Achern,DE",2022-04-07 10:32:48,25615
...,...,...,...,...,...,...,...,...
2122,2803655,Zusmarshausen,DE,10.60000,48.4000,"Zusmarshausen,DE",2022-04-07 10:32:48,6378
2123,2803588,Zwenkau,DE,12.31670,51.2167,"Zwenkau,DE",2022-04-07 10:32:48,9274
2124,2803558,Zwickau,DE,12.48330,50.7167,"Zwickau,DE",2022-04-07 10:32:48,89540
2125,2803537,Zwiesel,DE,13.23330,49.0167,"Zwiesel,DE",2022-04-07 10:32:48,9421


In [42]:
current_weather = pd.DataFrame()
minutely_weather = pd.DataFrame()
# only first to cites for testing
for index, city_row in cities.head(2).iterrows():
    API_KEY = "c21a13f270f9dfc33b7894b574d754ad"
    url = f"https://api.openweathermap.org/data/2.5/onecall?lat={city_row['city_latitude']}&lon={city_row['city_longitude']}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url).json()
        # Unpack weather list
        response["current"]["weather"] = response["current"]["weather"][0]
        city_current_weather = get_current_city_Weather(response["current"], city_row)
        current_weather = pd.concat([current_weather, city_current_weather])
        city_minutely_weather = get_minutely_city_Weather(response["minutely"], city_row)
        minutely_weather = pd.concat([minutely_weather, city_minutely_weather])
            
    except Exception as e:
        print(e)
        continue


In [30]:
def get_current_city_Weather(response_cur, city_row):
    current_weather_df = (
        pd.json_normalize(response_cur)
        .assign(municipality_country = city_row["municipality_country"])
    )            
    return current_weather_df

In [34]:
def get_minutely_city_Weather(response_min, city_row):
    minutely_city_weather_df = pd.DataFrame()
    for i in range(len(response_min)):
        minutely_city_weather = (
            pd.json_normalize(response_min)
            .assign(
                municipality_country = city_row["municipality_country"],
                in_minutes = i
                )
        )    
        minutely_city_weather_df = pd.concat([minutely_city_weather_df, minutely_city_weather])
    return minutely_city_weather_df

In [25]:
current_weather

""


In [44]:
minutely_weather.loc["municicpality_country"].sample(10)

,dt,precipitation,municipality_country,in_minutes
43,1649346360,0,"Aachen,DE",36
15,1649344680,0,"Aalen,DE",57
20,1649344980,0,"Aachen,DE",32
56,1649347140,0,"Aachen,DE",42
15,1649344680,0,"Aachen,DE",54
26,1649345340,0,"Aachen,DE",43
29,1649345520,0,"Aachen,DE",14
24,1649345220,0,"Aalen,DE",57
52,1649346900,0,"Aachen,DE",55
2,1649343900,0,"Aalen,DE",55
